# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

In [1]:
%%classpath add mvn
org.clustering4ever clustering4ever_2.11 0.9.6

In [2]:
%%classpath add mvn
org.apache.spark spark-core_2.11 2.4.3

# Clusterwise PLS

In [3]:
import org.clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS, ClusterwiseModel}
import scala.collection.{mutable, immutable}

import org.clustering4ever.spark.clustering.clusterwise.{Clusterwise, ClusterwiseCore, PLS, ClusterwiseModel}
import scala.collection.{mutable, immutable}


In [4]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf

val conf = new SparkConf().setAppName("Simple Application").set("spark.ui.enabled", "false")
val sc = new SparkContext("local[4]", "Simple Application", conf)

org.apache.spark.SparkContext@31b23501

In [8]:
%%bash
wget -P /tmp/ http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv

--2019-08-22 23:08:40--  http://www.clustering4ever.org/Datasets/Clusterwise/simdata.csv
Résolution de www.clustering4ever.org (www.clustering4ever.org)… 62.210.16.62
Connexion à www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 61840 (60K) [text/csv]
Sauvegarde en : « /tmp/simdata.csv »

     0K .......... .......... .......... .......... .......... 82%  241K 0s
    50K ..........                                            100%  499K=0,2s

2019-08-22 23:08:41 (265 KB/s) — « /tmp/simdata.csv » sauvegardé [61840/61840]




In [9]:
val dataXY = scala.io.Source.fromFile("/tmp/simdata.csv")
  .getLines
  .toSeq
  .map(_.split(',').map(_.toDouble))
  .map( a => (a.take(10), a.takeRight(3)) )
  .zipWithIndex
  .map{ case(xy,idx) => (idx, xy) }

[[(0,([D@29d57a71,[D@6627afbe)), (1,([D@45617bc5,[D@622213be)), (2,([D@3a42050d,[D@70c30934)), (3,([D@3eb2ebba,[D@4cffa031)), (4,([D@663faece,[D@edd9194)), (5,([D@271b54ce,[D@16549634)), (6,([D@d0603ee,[D@6f3562eb)), (7,([D@12f5f6e7,[D@20a227bb)), (8,([D@1d865932,[D@59605253)), (9,([D@649fd556,[D@55b1a981)), (10,([D@7a5125cc,[D@21f8021f)), (11,([D@3bd74613,[D@63f20d49)), (12,([D@10fe3bbe,[D@71772b44)), (13,([D@70911ca7,[D@3facfb9)), (14,([D@623837b3,[D@2add1dbb)), (15,([D@52515863,[D@783d297)), (16,([D@24a5f87b,[D@66d71991)), (17,([D@3e9dd768,[D@2793ad46)), (18,([D@2c7eefef,[D@723b7060)), (19,([D@22cddfd8,[D@188ebf5a)), (20,([D@43d43a33,[D@6a45ea59)), (21,([D@263adbd6,[D@4ba3d424)), (22,([D@29c47ac1,[D@6c164a82)), (23,([D@1c82d8bd,[D@49d53dfd)), (24,([D@43cd4474,[D@1dd144d5)), (25,([D@171c20e2,[D@7746cd29)), (26,([D@5e01c951,[D@49e1c9a8)), (27,([D@3f0cfb46,[D@7e62560b)), (28,([D@2ef953d7,[D@69926329)), (29,([D@311c7396,[D@5da2a9b0)), (30,([D@39a12e38,[D@43d0890e)), (31,([D@1182d221,[D@

## Run a full Clusterwise
### Parameters

In [10]:
val init = 20
val nbCV = 10
val g = 4
val h = 2
val sizeBloc = 10
val kPred = 25
val nbLaunch = 1
val standardization = true
val withY = false

false

In [11]:
val t0 = System.currentTimeMillis
val (sqrmseCalAndVal, _) = Clusterwise.run(sc, dataXY, g, h, nbCV, init, kPred, withY, standardization, sizeBloc)
val t1 = System.currentTimeMillis

(t1 - t0) / 1000D

org.apache.spark.SparkException:  Task not serializable

In [12]:
sqrmseCalAndVal

<console>: 98

## Run one Initialization of Clusterwise

In [ ]:
val (dsPerClass, predFitted, coIntercept, coXYcoef, critReg, mapsRegCrit, classedReg) = ClusterwiseCore.plsPerDot(dataXY, h, g)

## Run one PLS regression

In [ ]:
val inputX = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => (id, x) }:_*)
val inputY = mutable.ArrayBuffer(dataXY.map{ case (id, (x, y)) => y }:_*)

PLS.runPLS(inputX, inputY, 2)._2